## Step 1: Load Phase 3 Results

In [1]:
import pandas as pd
import json
from datetime import datetime

print("="*70)
print("FINAL SUBMISSION GENERATOR - CORRECT FORMAT")
print("="*70)

# Load Phase 3 results
try:
    phase3_results = pd.read_csv(r'C:\Users\omgha\OneDrive\Documents\GitHub\Libaspace-AI-intern-challenge\Phase3\logistic_regression_results.csv')
    print(f"\nLoaded Phase 3 results: {len(phase3_results)} fields")
except FileNotFoundError:
    print("\nCould not find phase3_with_logistic_regression.csv")
    phase3_results = None

if phase3_results is not None:
    # Load original dataset for labels
    try:
        dataset = pd.read_csv(r'C:\Users\omgha\OneDrive\Documents\GitHub\Libaspace-AI-intern-challenge\Phase2\dataset_with_rules.csv')
        print(f"Loaded dataset: {len(dataset)} fields")
        has_labels = True
    except:
        print("⚠️  Could not find dataset for labels")
        dataset = None
        has_labels = False
    
    # Summary
    print(f"\n Accuracy: {(phase3_results['lr_correct'].mean()*100):.1f}%")

FINAL SUBMISSION GENERATOR - CORRECT FORMAT

Loaded Phase 3 results: 23 fields
Loaded dataset: 23 fields

 Accuracy: 87.0%


## Step 2: Generate Evidence for Each Prediction

In [2]:
def generate_evidence(row):
    """
    Generate evidence/reasoning for each prediction
    """
    evidence = []
    
    # DOM-based evidence
    if row['has_options'] == 1:
        evidence.append(f"Options detected ({int(row['options_count'])} options)")
    else:
        evidence.append("No options in raw form")
    
    # Semantic evidence
    if row['is_yes_no_question'] == 1:
        evidence.append("Yes/no question pattern")
    
    if row['is_required'] == 1:
        evidence.append("Required field")
    
    # ML confidence evidence
    confidence = row['lr_confidence']
    if confidence >= 0.95:
        evidence.append("High confidence (LR ≥ 0.95)")
    elif confidence >= 0.85:
        evidence.append("Medium-high confidence (LR ≥ 0.85)")
    elif confidence >= 0.70:
        evidence.append("Medium confidence (LR ≥ 0.70)")
    else:
        evidence.append("Lower confidence (LR < 0.70)")
    
    # Prediction type
    pred = str(row['lr_prediction']).lower()
    if pred == 'select':
        if row['has_options'] == 1:
            evidence.append("LR predicts: SELECT (based on options)")
        elif row['is_yes_no_question'] == 1:
            evidence.append("LR predicts: SELECT (despite no visible options)")
        else:
            evidence.append("LR predicts: SELECT")
    else:
        evidence.append("LR predicts: TEXT")
    
    return evidence

if phase3_results is not None:
    # Test evidence generation
    test_row = phase3_results.iloc[0]
    test_evidence = generate_evidence(test_row)
    
    print("\n" + "="*70)
    print("EVIDENCE GENERATION TEST")
    print("="*70)
    print(f"\nField: {test_row['field_id']}")
    print(f"Prediction: {test_row['lr_prediction']}")
    print(f"Confidence: {test_row['lr_confidence']:.3f}")
    print(f"\nGenerated Evidence:")
    for i, ev in enumerate(test_evidence, 1):
        print(f"  {i}. {ev}")


EVIDENCE GENERATION TEST

Field: first_name
Prediction: text
Confidence: 0.807

Generated Evidence:
  1. No options in raw form
  2. Required field
  3. Medium confidence (LR ≥ 0.70)
  4. LR predicts: TEXT


## Step 3: Generate predictions.jsonl

In [3]:
if phase3_results is not None:
    print("\n" + "="*70)
    print("GENERATING: predictions.jsonl")
    print("="*70)
    
    predictions_list = []
    
    for idx, row in phase3_results.iterrows():
        # Generate evidence
        evidence = generate_evidence(row)
        
        # Create prediction object
        prediction = {
            "id": str(row['field_id']),
            "label": str(row['field_id']),  # Using field_id as label
            "input_kind_pred": str(row['lr_prediction']).lower(),
            "confidence": float(round(row['lr_confidence'], 2)),
            "evidence": evidence
        }
        predictions_list.append(prediction)
    
    # Save as JSONL
    with open('predictions.jsonl', 'w') as f:
        for pred in predictions_list:
            f.write(json.dumps(pred) + '\n')
    
    print(f"\n✅ Saved: predictions.jsonl")
    print(f"   Lines: {len(predictions_list)}")
    print(f"   Format: One JSON object per line")
    
    # Show first 3 predictions
    print(f"\n📋 First 3 predictions:")
    for i, pred in enumerate(predictions_list[:3]):
        print(f"\n{i+1}. {pred['id']}")
        print(f"   Prediction: {pred['input_kind_pred']}")
        print(f"   Confidence: {pred['confidence']}")
        print(f"   Evidence: {len(pred['evidence'])} items")
        for ev in pred['evidence']:
            print(f"     - {ev}")


GENERATING: predictions.jsonl

✅ Saved: predictions.jsonl
   Lines: 23
   Format: One JSON object per line

📋 First 3 predictions:

1. first_name
   Prediction: text
   Confidence: 0.81
   Evidence: 4 items
     - No options in raw form
     - Required field
     - Medium confidence (LR ≥ 0.70)
     - LR predicts: TEXT

2. last_name
   Prediction: text
   Confidence: 0.82
   Evidence: 4 items
     - No options in raw form
     - Required field
     - Medium confidence (LR ≥ 0.70)
     - LR predicts: TEXT

3. email
   Prediction: text
   Confidence: 0.85
   Evidence: 4 items
     - No options in raw form
     - Required field
     - Medium-high confidence (LR ≥ 0.85)
     - LR predicts: TEXT


## Step 4: Also Generate predictions.json (JSON Array Format)

In [4]:
if phase3_results is not None:
    print("\n" + "="*70)
    print("GENERATING: predictions.json (JSON Array Format)")
    print("="*70)
    
    # Save as JSON array
    with open('predictions.json', 'w') as f:
        json.dump(predictions_list, f, indent=2)
    
    print(f"\n✅ Saved: predictions.json")
    print(f"   Format: JSON array")
    print(f"   Fields: {len(predictions_list)}")


GENERATING: predictions.json (JSON Array Format)

✅ Saved: predictions.json
   Format: JSON array
   Fields: 23


## Step 5: Verify Output Quality

In [5]:
if phase3_results is not None:
    print("\n" + "="*70)
    print("QUALITY VERIFICATION")
    print("="*70)
    
    # Verify counts
    print(f"\n✅ Count Checks:")
    print(f"   Total predictions: {len(predictions_list)}")
    print(f"   Expected: 23")
    print(f"   Match: {len(predictions_list) == 23}")
    
    # Verify format
    print(f"\n✅ Format Checks:")
    
    all_valid = True
    for i, pred in enumerate(predictions_list):
        # Check required fields
        if not all(k in pred for k in ['id', 'label', 'input_kind_pred', 'confidence', 'evidence']):
            print(f"   ❌ Prediction {i}: Missing required fields")
            all_valid = False
        
        # Check prediction value
        if pred['input_kind_pred'] not in ['text', 'select']:
            print(f"   ❌ Prediction {i}: Invalid input_kind_pred: {pred['input_kind_pred']}")
            all_valid = False
        
        # Check confidence range
        if not (0.0 <= pred['confidence'] <= 1.0):
            print(f"   ❌ Prediction {i}: Confidence out of range: {pred['confidence']}")
            all_valid = False
        
        # Check evidence
        if not isinstance(pred['evidence'], list) or len(pred['evidence']) == 0:
            print(f"   ❌ Prediction {i}: Invalid evidence")
            all_valid = False
    
    if all_valid:
        print(f"   ✅ All predictions have correct format")
    
    # Confidence distribution
    confidences = [p['confidence'] for p in predictions_list]
    print(f"\n✅ Confidence Distribution:")
    print(f"   Mean: {sum(confidences)/len(confidences):.3f}")
    print(f"   Min: {min(confidences):.3f}")
    print(f"   Max: {max(confidences):.3f}")
    
    # Prediction distribution
    text_count = sum(1 for p in predictions_list if p['input_kind_pred'] == 'text')
    select_count = sum(1 for p in predictions_list if p['input_kind_pred'] == 'select')
    print(f"\n✅ Prediction Distribution:")
    print(f"   TEXT: {text_count}")
    print(f"   SELECT: {select_count}")


QUALITY VERIFICATION

✅ Count Checks:
   Total predictions: 23
   Expected: 23
   Match: True

✅ Format Checks:
   ✅ All predictions have correct format

✅ Confidence Distribution:
   Mean: 0.823
   Min: 0.540
   Max: 1.000

✅ Prediction Distribution:
   TEXT: 14
   SELECT: 9


## Step 6: Final Checklist

In [6]:
if phase3_results is not None:
    print("\n" + "="*70)
    print("FINAL CHECKLIST")
    print("="*70)
    
    checklist = f"""
✅ SUBMISSION REQUIREMENTS MET:
  ✅ 23 field predictions (100% coverage)
  ✅ Correct JSON format (id, label, input_kind_pred, confidence, evidence)
  ✅ Predictions are 'text' or 'select' (lowercase)
  ✅ Confidence scores 0.0-1.0
  ✅ Evidence provided for each prediction
  ✅ Valid JSON/JSONL format
  
📁 OUTPUT FILES CREATED:
  ✅ predictions.jsonl (RECOMMENDED)
     - One JSON object per line
     - File size: {round(__import__('os').path.getsize('predictions.jsonl')/1024, 1)} KB
  
  ✅ predictions.json (ALTERNATIVE)
     - JSON array format
     - File size: {round(__import__('os').path.getsize('predictions.json')/1024, 1)} KB

🎯 APPROACH:
  - Phase 1: Baseline (73.9%)
  - Phase 2: Rule-Based (87.0%)
  - Phase 3: Logistic Regression (87.0%) ← FINAL
  - Phase 4: Skipped (DOM incomplete)
  
📊 FINAL ACCURACY: {(phase3_results['lr_correct'].mean()*100):.1f}%

🚀 READY TO SUBMIT!
   Use: predictions.jsonl
   
Checklist:
  ✅ Schema matches PDF requirement
  ✅ All fields included
  ✅ Evidence provided
  ✅ Valid JSON format
  ✅ Ready for submission
    """
    
    print(checklist)
    
    print(f"\n" + "="*70)
    print(f"✅ SUBMISSION READY!")
    print(f"="*70)
    print(f"\n📤 Submit: predictions.jsonl")
    print(f"\nFile is in the current directory and ready to upload!")


FINAL CHECKLIST

✅ SUBMISSION REQUIREMENTS MET:
  ✅ 23 field predictions (100% coverage)
  ✅ Correct JSON format (id, label, input_kind_pred, confidence, evidence)
  ✅ Predictions are 'text' or 'select' (lowercase)
  ✅ Confidence scores 0.0-1.0
  ✅ Evidence provided for each prediction
  ✅ Valid JSON/JSONL format
  
📁 OUTPUT FILES CREATED:
  ✅ predictions.jsonl (RECOMMENDED)
     - One JSON object per line
     - File size: 5.4 KB
  
  ✅ predictions.json (ALTERNATIVE)
     - JSON array format
     - File size: 6.9 KB

🎯 APPROACH:
  - Phase 1: Baseline (73.9%)
  - Phase 2: Rule-Based (87.0%)
  - Phase 3: Logistic Regression (87.0%) ← FINAL
  - Phase 4: Skipped (DOM incomplete)
  
📊 FINAL ACCURACY: 87.0%

🚀 READY TO SUBMIT!
   Use: predictions.jsonl
   
Checklist:
  ✅ Schema matches PDF requirement
  ✅ All fields included
  ✅ Evidence provided
  ✅ Valid JSON format
  ✅ Ready for submission
    

✅ SUBMISSION READY!

📤 Submit: predictions.jsonl

File is in the current directory and ready